<a href="https://colab.research.google.com/github/eduval91/ED91/blob/main/AssessmentTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from bs4 import BeautifulSoup

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("darinhawley/us-top-1000-baby-names-18802020")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/us-top-1000-baby-names-18802020


In [3]:
path = "/root/.cache/kagglehub/datasets/darinhawley/us-top-1000-baby-names-18802020/versions/1"
files = os.listdir(path)
print(files)

['babynames1880-2020.csv']


In [4]:
csv_path = os.path.join(path, "babynames1880-2020.csv")
df = pd.read_csv(csv_path)
df.head()

,Year,Name,Gender,Births,Rank
0,2020,Liam,M,19659,1
1,2020,Noah,M,18252,2
2,2020,Oliver,M,14147,3
3,2020,Elijah,M,13034,4
4,2020,William,M,12541,5


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281877 entries, 0 to 281876
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Year    281877 non-null  int64 
 1   Name    281877 non-null  object
 2   Gender  281877 non-null  object
 3   Births  281877 non-null  int64 
 4   Rank    281877 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 10.8+ MB


##Seperate 2007 and 2020 baby name data




In [6]:
df_2007 = df[df['Year'] == 2007].copy()
df_2020 = df[df['Year'] == 2020].copy()

##Rank names by gender & births

In [7]:
df_2007['Rank'] = df_2007.groupby('Gender')['Births'].rank(ascending=False, method='min')
df_2020['Rank'] = df_2020.groupby('Gender')['Births'].rank(ascending=False, method='min')

##Identify top 100 names in 2007

In [8]:
top_2007 = df_2007[df_2007['Rank'] <= 100]
top_2007_keys = top_2007[['Name','Gender']]

##Merge with 2020 data to compare 2007 names and their rank

In [9]:
merged = pd.merge(top_2007_keys, df_2020, on=['Name', 'Gender'], how='left', suffixes=('2007','2020'))
merged

,Name,Gender,Year,Births,Rank
0,Jacob,M,2020,9196,15.0
1,Michael,M,2020,9717,12.0
2,Ethan,M,2020,9464,13.0
3,Joshua,M,2020,5924,53.0
4,Daniel,M,2020,9419,14.0
...,...,...,...,...,...
195,Gracie,F,2020,1368,209.0
196,Molly,F,2020,1635,174.0
197,Valeria,F,2020,1803,150.0
198,Caroline,F,2020,3345,71.0


## Find names that are outside the top 100 sorted by decline

In [10]:
declined = merged[(merged['Rank'] > 100) | (merged['Rank'].isna())]

In [11]:
declined = merged[(merged['Rank'] > 100) | (merged['Rank'].isna())]
top10_male = declined[declined['Gender'] == 'M'].sort_values(by='Rank').head(10)
top10_female = declined[declined['Gender'] == 'F'].sort_values(by='Rank').head(10)

print("Top 10 male names that fell out of top 100:")
print(top10_male[['Name', 'Rank']])

print("\nTop 10 female names that fell out of top 100:")
print(top10_female[['Name', 'Rank']])

Top 10 male names that fell out of top 100:
         Name   Rank
39       Evan  105.0
63    Brayden  111.0
58      Jason  119.0
78      Chase  123.0
56      Diego  125.0
81  Nathaniel  126.0
20      Tyler  129.0
83       Cole  131.0
41    Zachary  135.0
59       Luis  137.0

Top 10 female names that fell out of top 100:
          Name   Rank
180    Melanie  105.0
133      Julia  107.0
152      Maria  109.0
135     Kaylee  117.0
164  Mackenzie  119.0
178   Isabelle  121.0
169    Arianna  123.0
192       Mary  124.0
162      Kylie  131.0
138  Katherine  135.0
